## Extract and save experimental atomic energy levels

In [1]:
import pandas as pd
import numpy as np
import os, re

In [2]:
# Specify the atom and the charge state
atom = 'Rh'
charge = 0

In [3]:
astr = atom + '+' + 'i' * (charge + 1)
url = f'https://physics.nist.gov/cgi-bin/ASD/energy1.pl?de=0&spectrum={astr}&submit=Retrieve+Data&units=0&format=0&output=0&page_size=15&multiplet_ordered=0&conf_out=on&term_out=on&level_out=on&unc_out=1&j_out=on&lande_out=on&perc_out=on&biblio=on&temp='

In [4]:
# Read all the tables
dfs = pd.read_html(url, header=0)

In [5]:
# Select the table that contains spectroscopic data
for df in dfs:
    if 'Configuration' in df.columns:
        dfexpt = df

In [6]:
# Keep selected columns
ecol = 'Level (cm-1)'
cols = ['Configuration', 'Term', 'J', ecol]
dfexpt = dfexpt[cols]

In [7]:
# Remove spaces from numbers and convert to float
for irow, estr in enumerate(dfexpt[ecol]):
    try:
        estr = str(estr)  # in case it has already been converted to a number
        e = float(re.sub('\s', '', estr.strip()))
    except ValueError:
        # energy might have "?" or other qualifier; replace with Nan
        print(f'** removing non-numeric energy value "{estr}"')
        e = np.nan
    dfexpt.at[irow, ecol] = e

In [8]:
# Discard rows that lack energies
dfexpt = dfexpt[dfexpt[ecol].notna()]
# Re-index to make continuous
dfexpt = dfexpt.reset_index(drop=True)
display(dfexpt)

,Configuration,Term,J,Level (cm-1)
0,4d8(3F)5s,a 4F,9/2,0.0
1,NaN,NaN,7/2,1529.97
2,NaN,NaN,5/2,2598.03
3,NaN,NaN,3/2,3472.68
4,4d9,a 2D,5/2,3309.86
...,...,...,...,...
131,NaN,26,"1/2,3/2",52473.55
132,NaN,32°,"3/2,5/2",53599.28
133,NaN,27,3/2,56087.6
134,NaN,28,"3/2,5/2",57610.48


In [9]:
# Where Configuration or Term is NaN, replace it with value from preceding row
for i, row in dfexpt.iterrows():
    for col in ['Configuration', 'Term']:
        if str(row[col]).lower() == 'nan':
            dfexpt.loc[i, col] = dfexpt.loc[i-1, col]

In [10]:
display(dfexpt)

,Configuration,Term,J,Level (cm-1)
0,4d8(3F)5s,a 4F,9/2,0.0
1,4d8(3F)5s,a 4F,7/2,1529.97
2,4d8(3F)5s,a 4F,5/2,2598.03
3,4d8(3F)5s,a 4F,3/2,3472.68
4,4d9,a 2D,5/2,3309.86
...,...,...,...,...
131,4d8(1G)5p,26,"1/2,3/2",52473.55
132,4d8(1G)5p,32°,"3/2,5/2",53599.28
133,4d8(1G)5p,27,3/2,56087.6
134,4d8(1G)5p,28,"3/2,5/2",57610.48


In [11]:
# Save to Excel file
fxl = atom + '_' + 'I' * (charge + 1) + '_exptl_levels.xlsx'
if os.path.isfile(fxl):
    print(f'File {fxl} already exists!')
else:
    dfexpt.to_excel(fxl, index=False)
    print(f'Data saved to file {fxl}')

Data saved to file Rh_I_exptl_levels.xlsx


In [26]:
# Get naive value of SOC for low terms
import chem_subs as chem

print(f'Naive values of E_so for low terms of {atom}')
ecol = 'Level (cm-1)'
terms = []
for t in dfexpt.Term:
    if t not in terms:
        terms.append(t)
for term in terms[:3]:
    S, L = chem.SL_from_term(term)
    mult = (2*S + 1) * (2*L + 1)
    subdf = dfexpt[dfexpt.Term == term].copy()
    subdf['g'] = 2 * subdf['J'].apply(chem.halves_to_float) + 1
    if subdf.g.sum() != mult:
        print(f'*** Total multiplicity should be {mult} but levels provide {subdf.g.sum()}')
    else:
        SOC = -1 * np.dot(subdf[ecol], subdf.g) / mult
        print(f'E_so({term}) = {SOC:.1f} cm-1 from eq. (1)')
    #display(subdf)

Naive values of E_so for low terms of Rh
E_so(a 4F) = -1490.0 cm-1 from eq. (1)
E_so(a 2D) = -4249.1 cm-1 from eq. (1)
E_so(a 2F) = -6591.1 cm-1 from eq. (1)
